In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
!pip install lifelines
from lifelines import WeibullAFTFitter, LogNormalAFTFitter

file_path = "/kaggle/input/lungss/lung.csv"
df = pd.read_csv(file_path)

# Clean column names
df.columns = [c.strip().lower().replace(".", "_") for c in df.columns]

# Prepare variables
df["event"] = (df["status"] == 1).astype(int)
df["sex_male"] = (df["sex"] == 1).astype(int)
df["inst"] = df["inst"].astype(str)
inst_dummies = pd.get_dummies(df["inst"], prefix="inst", drop_first=True)
df2 = pd.concat([df, inst_dummies], axis=1)

# Covariates
covariates = [
    "age",
    "sex_male",
    "ph_ecog",
    "ph_karno",
    "pat_karno",
    "meal_cal",
    "wt_loss",
] + list(inst_dummies.columns)

model_df = df2[["time", "event"] + covariates].dropna() #all null values are dropped


weib = WeibullAFTFitter().fit(model_df, duration_col="time", event_col="event")
logn = LogNormalAFTFitter().fit(model_df, duration_col="time", event_col="event")

# Compute AICs
weib_aic, logn_aic = weib.AIC_, logn.AIC_

# Compute survival at 188 days for specified patient
patient = {c: 0 for c in covariates}
patient.update({
    "age": 68,
    "sex_male": 1,
    "ph_ecog": 1,
    "ph_karno": 80,
    "pat_karno": 85,
    "meal_cal": 1000,
    "wt_loss": 10,
})
patient["inst_3"] = 1

patient_df = pd.DataFrame([patient])[covariates]

t = 188
weib_sf = weib.predict_survival_function(patient_df, times=[t]).iloc[0, 0]
logn_sf = logn.predict_survival_function(patient_df, times=[t]).iloc[0, 0]

weib.print_summary()

results = {
    "Weibull_AIC": weib_aic,
    "LogNormal_AIC": logn_aic,
    "Weibull_Survival_188": weib_sf,
    "LogNormal_Survival_188": logn_sf,
}
results


/usr/local/lib/python3.11/dist-packages/lifelines/fitters/__init__.py:2086: ApproximationWarning: The Hessian was not invertible. We will instead approximate it using the pseudo-inverse.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

Some ways to possible ways fix this:

0. Are there any lifelines warnings outputted during the `fit`?
1. Does a particularly large variable need to be centered to 0?
2. Inspect your DataFrame: does everything look as expected? Do you need to add/drop a constant (intercept) column?
3. Is there high-collinearity in the dataset? Try using the variance inflation factor (VIF) to find redundant variables.
4. Trying adding a small penalizer (or changing it, if already present). Example: `WeibullAFTFitter(penalizer=0.01).fit(...)`.
5. Are there any extreme outliers? Try modeling them or dropping them to see if it helps convergence.

  warnings.warn(warning_text, exceptions.ApproximationWarning)
/usr/local/l

<lifelines.WeibullAFTFitter: fitted with 168 total observations, 121 right-censored observations>
             duration col = 'time'
                event col = 'event'
   number of observations = 168
number of events observed = 47
           log-likelihood = -344.24
         time fit was run = 2025-10-20 09:46:00 UTC

---
                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                   
lambda_ age         0.01      1.01      0.01           -0.01            0.03                0.99                1.03
        inst_10.0 225.43  7.98e+97      0.00          225.43          225.43            7.98e+97            7.98e+97
        inst_11.0  -0.46      0.63      0.29           -1.02            0.10                0.36                1.11
        inst_12.0  -0.08      0.93      0.30           -0.66            0.51                0.52                1.66
        inst_13.0  -0.10      0.91      0.26           -0.61            0.42                0.54                1.52
        inst_15.0  -0.02      0.98      0.39           -0.78            0.75                0.46                2.11
        inst_16.0  -0.04      0.96      0.31           -0.64            0.56                0.53                1.76
        inst_2.0   -0.29      0.75      0.49           -1.25            0.67                0.29                1.95
        inst_21.0 183.39  4.42e+79      0.00          183.39          183.39            4.42e+79            4.42e+79
        inst_22.0   0.23      1.26      0.33           -0.41            0.88                0.66                2.40
        inst_26.0  -0.42      0.66      0.39           -1.18            0.34                0.31                1.40
        inst_3.0   -0.00      1.00      0.32           -0.63            0.62                0.53                1.86
        inst_32.0  -0.80      0.45      0.30           -1.38           -0.22                0.25                0.81
        inst_33.0   6.56    703.58      0.00            6.56            6.56              703.58              703.58
        inst_4.0  292.92 1.63e+127      0.00          292.92          292.92           1.63e+127           1.63e+127
        inst_5.0    0.47      1.60      0.50           -0.51            1.45                0.60                4.27
        inst_6.0   -0.00      1.00      0.37           -0.73            0.72                0.48                2.06
        inst_7.0   -0.32      0.73      0.39           -1.08            0.45                0.34                1.56
        inst_nan  193.09  7.20e+83      0.00          193.09          193.09            7.20e+83            7.20e+83
        meal_cal    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
        pat_karno   0.00      1.00      0.01           -0.01            0.02                0.99                1.02
        ph_ecog     0.08      1.09      0.21           -0.33            0.49                0.72                1.64
        ph_karno   -0.01      0.99      0.01           -0.03            0.01                0.97                1.01
        sex_male    0.15      1.16      0.17           -0.18            0.47                0.83                1.61
        wt_loss     0.00      1.00      0.01           -0.01            0.01                0.99                1.01
        Intercept   6.03    416.61      1.36            3.36            8.70               28.92             6001.72
rho_    Intercept   0.81      2.24      0.11            0.59            1.02                1.80                2.78

                   cmp to                    z      p  -log2(p)
param   covariate                                              
lambda_ age          0.00                 0.97   0.33      1.59
        inst_10.0    0.00  4722100911505220.00 <0.005       inf
        inst_11.0    0.00

{'Weibull_AIC': 742.4835209906133,
 'LogNormal_AIC': 736.5097037977712,
 'Weibull_Survival_188': 0.9684589812703487,
 'LogNormal_Survival_188': 0.9873319227095851}